In [7]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import numpy
import pandas
import seaborn
import tensorflow
from scipy import stats

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Reshape, GlobalAveragePooling1D, Conv1D, MaxPooling1D
from keras import utils

from keras.models import load_model

In [2]:
def feature_normalize(dataset):

    mu = np.mean(dataset, axis=0)
    sigma = np.std(dataset, axis=0)
    return (dataset - mu)/sigma

# Not used right now
def feature_normalize(dataset):

    mu = np.mean(dataset, axis=0)
    sigma = np.std(dataset, axis=0)
    return (dataset - mu)/sigma

def plot_axis(ax, x, y, title):

    ax.plot(x, y)
    ax.set_title(title)
    ax.xaxis.set_visible(False)
    ax.set_ylim([min(y) - np.std(y), max(y) + np.std(y)])
    ax.set_xlim([min(x), max(x)])
    ax.grid(True)

def plot_kelas(kelas, data):

    fig, (ax0) = plt.subplots(nrows=1,
         figsize=(15, 10),
         sharex=True)
    plot_axis(ax0, data['id'], data['VMPressure'], 'VMPressure')
    plt.subplots_adjust(hspace=0.2)
    fig.suptitle(kelas)
    plt.subplots_adjust(top=0.90)
    plt.show()

def create_segments_and_labels(df, time_steps, step, label_name):
    # x, y, z acceleration as features
    N_FEATURES = 1
    # Number of steps to advance in each iteration (for me, it should always
    # be equal to the time_steps in order to have no overlap between segments)
    # step = time_steps
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
        xs = df['vm_pressure'].values[i: i + time_steps]
        # Retrieve the most often used label in this segment
        # Check if the mode is a scalar and handle it accordingly
        mode_result = stats.mode(df[label_name][i: i + time_steps])
        # Fix: Check if mode_result.mode is a scalar or an array
        label = mode_result.mode if np.isscalar(mode_result.mode) else mode_result.mode[0]
        segments.append([xs])
        labels.append(label)



        # label = stats.mode(df[label_name][i: i + time_steps]).mode[0]
        # segments.append([xs])
        # labels.append(label)

    # Bring the segments into a better shape
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, N_FEATURES)
    labels = np.asarray(labels)

    return reshaped_segments, labels

In [3]:
# Set some standard parameters upfront
pd.options.display.float_format = '{:.1f}'.format
sns.set() # Default seaborn look and feel
plt.style.use('ggplot')
print('keras version ', keras.__version__)


LABELS = ['VM', 'Tanpa VM']
TIME_PERIODS = 100
STEP_DISTANCE = 1
BATCH_SIZE = 400
EPOCHS = 50
LABEL_CLASS_ENCODED = 'class_encoded'

keras version  3.7.0


In [4]:
df = pd.read_csv('dataset.csv', header=None, names=['id', 'class', 'vm_pressure'])

df.head()

,id,class,vm_pressure
0,10,1,125
1,10,1,130
2,10,1,155
3,10,1,143
4,10,1,108


In [5]:
encoder = preprocessing.LabelEncoder()
df[LABEL_CLASS_ENCODED] = encoder.fit_transform(df['class'].values.ravel())

df_test = df[df['id'] > 20]
df_train = df[df['id'] <= 20]

df_train['vm_pressure'] = feature_normalize(df['vm_pressure'])
x_train, y_train = create_segments_and_labels(df_train, TIME_PERIODS, STEP_DISTANCE, LABEL_CLASS_ENCODED)

C:\Users\atmatech\AppData\Local\Temp\ipykernel_9220\983505379.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['vm_pressure'] = feature_normalize(df['vm_pressure'])


In [6]:
print(x_train[0])

[[-0.7836026 ]
 [-0.75649536]
 [-0.62095916]
 [-0.6860165 ]
 [-0.8757672 ]
 [-0.85950285]
 [-0.6860165 ]
 [-0.45831573]
 [-0.13845034]
 [ 0.13804346]
 [ 0.26273677]
 [ 0.33863702]
 [ 0.41453728]
 [ 0.4633303 ]
 [ 0.41453728]
 [ 0.40911585]
 [ 0.66392386]
 [ 0.78861713]
 [ 0.83741015]
 [ 0.9621035 ]
 [ 1.0217394 ]
 [ 1.0434252 ]
 [ 1.0651109 ]
 [ 1.0759538 ]
 [ 1.1030611 ]
 [ 1.113904  ]
 [ 1.0705324 ]
 [ 1.0434252 ]
 [ 1.0759538 ]
 [ 1.0705324 ]
 [ 1.005475  ]
 [ 1.0271609 ]
 [ 1.0108964 ]
 [ 1.0108964 ]
 [ 1.0108964 ]
 [ 1.005475  ]
 [ 1.0000536 ]
 [ 0.956682  ]
 [ 0.9946321 ]
 [ 0.956682  ]
 [ 0.9783678 ]
 [ 0.9892107 ]
 [ 0.98378927]
 [ 0.98378927]
 [ 0.9783678 ]
 [ 0.9458391 ]
 [ 0.8645174 ]
 [ 0.75066704]
 [ 0.701874  ]
 [ 0.6910311 ]
 [ 0.72898126]
 [ 0.7235598 ]
 [ 0.7235598 ]
 [ 0.6801882 ]
 [ 0.6693453 ]
 [ 0.65308094]
 [ 0.66392386]
 [ 0.69645256]
 [ 0.62597376]
 [ 0.56091636]
 [ 0.6042879 ]
 [ 0.59344506]
 [ 0.6097094 ]
 [ 0.6042879 ]
 [ 0.5988665 ]
 [ 0.5771807 ]
 [ 0.55549

In [10]:
model = load_model('model.h5')

# model.summary()

In [14]:
print(model.predict(numpy.array([x_train[0]])))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
[[0.00519458 0.9989166 ]]
